# 811s
### outputs
* Marked as completed in smart sheets
* Marked as not completed in smart sheets

In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
path = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type = ".xlsx"
load_it = glob.glob(path + file_type)
df = pd.read_excel(max(load_it, key=os.path.getctime))

In [3]:
checks = df.copy()

In [4]:
# set up 
checks_811s = checks[['APN_ROW Segment','Street #', 'Street Name', 'County','811 Called (A&M)',
                      'Site Assessment', "Debris Crew", "Not Eligible", 'Withdrawal']]
checks_811s = checks_811s[(checks_811s['Not Eligible'].isnull()) & (checks_811s['Withdrawal'].isnull())]
checks_811s = checks_811s.copy()
checks_811s['811 Called (A&M)'].fillna(0, inplace=True)

In [5]:
# need to replace floats with what is means
checks_811s.replace(1.0, "Complete", inplace=True)
checks_811s.replace(0.0, "Not done", inplace=True)

In [6]:
# set up our df for each catagory
checks_811s_notdone = checks_811s[(checks_811s['811 Called (A&M)'] == "Not done")]
checks_811s_complete= checks_811s[(checks_811s['811 Called (A&M)']=="Complete")] 


# output for excles
checks_811s_notdone.to_excel("811s (A&M) Not completed.xlsx")
checks_811s_complete.to_excel("811s (A&M) completed.xlsx")